In [1]:
import mocalum as mc
import numpy as np

koshava_xyz = np.array([500, 500, 0])
no_scans = 1000
# wind field setup
ref_height = 100      # power law reference height
meas_height = 100 
shear_exponent = 0.2  # power law shear exponent
wind_speed = 8       # wind speed at referenec height
wind_dir = 20 # wind direction
w = 0 

# beam steering setup
distance = 1000   # meter
elevation = np.degrees(np.arcsin(meas_height / distance))
angular_res = 1   # degree
azimuth_mid = 45  # central azimuth angle
sector_size = 30  # degree
scan_speed = 1    # degree.s^-1
max_speed = 50    # degree.s^-1
max_acc = 100     # degree.s^-2

# Uncertainty terms
no_sim = 10000 # number of simulations
corr_coef = 0  # correlation coefficient
mu = 0         # we assume no systematic uncertainty
azim_std = 0.1 # degrees
elev_std = 0.1 # degrees
dis_std = 10   # meters
rad_std = 0.1  # m.s-1  In [23]:  
pl_mc = mc.Mocalum()
pl_mc.add_lidar('sterenn', koshava_xyz)
pl_mc.add_lidar('koshava', koshava_xyz)

pl_mc.set_ivap_probing('koshava', sector_size*2, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)

pl_mc.set_ivap_probing('sterenn', sector_size*2, azimuth_mid, angular_res, elevation, distance, 
                    no_scans, scan_speed, max_speed,max_acc)

In [2]:
pl_mc.gen_unc_contributors('sterenn')

In [10]:
pl_mc.data.probing['sterenn'].unc_est.mean()

<xarray.DataArray 'unc_est' ()>
array(-0.00044278)

In [3]:
pl_mc.data.ffield_bbox_cfg

{'koshava': {'CRS': {'x': 'Absolute coordinate, coresponds to Easting in m',
   'y': 'Absolute coordinate, coresponds to Northing in m',
   'z': 'Absolute coordinate, coresponds to height above sea level in m',
   'rot_matrix': array([[ 1.,  0.],
          [-0.,  1.]])},
  'x': {'min': 732.5216983609398,
   'max': 1486.0840623344543,
   'offset': 0,
   'res': 25},
  'y': {'min': 732.5216983609398,
   'max': 1486.0840623344543,
   'offset': 0,
   'res': 25},
  'z': {'min': 95.0, 'max': 105.0, 'offset': 0, 'res': 5},
  't': {'min': 0.0, 'max': 61698.3, 'offset': 0, 'res': 1.0114641223626617}},
 'sterenn': {'CRS': {'x': 'Absolute coordinate, coresponds to Easting in m',
   'y': 'Absolute coordinate, coresponds to Northing in m',
   'z': 'Absolute coordinate, coresponds to height above sea level in m',
   'rot_matrix': array([[ 1.,  0.],
          [-0.,  1.]])},
  'x': {'min': 732.5216983609398,
   'max': 1486.0840623344543,
   'offset': 0,
   'res': 25},
  'y': {'min': 732.5216983609398,


In [7]:
pl_mc.data.probing['koshava'][pl_mc.data.tmp_unc].values = pl_mc.data.tmp_unc_val

In [22]:
i = _get_index(pl_mc.data.probing, 'koshava')

pl_mc.data.probing.x[i] = pl_mc.x + 1000
pl_mc.data.probing['x'][i]

<xarray.DataArray 'x' (lidar_id: 1, time: 61000)>
array([[1757.52169836, 1774.25570624, 1790.90617316, ..., 2451.51121823,
        2456.44331123, 2461.08406233]])
Coordinates:
  * lidar_id  (lidar_id) object 'koshava'
  * time      (time) float64 0.0 1.0 2.0 3.0 ... 6.17e+04 6.17e+04 6.17e+04
Attributes:
    long_name:      X coordinate
    standard_name:  projection_x_coordinate
    units:          m
    URI:

In [14]:
pl_mc.data.probing['x'][0]

<xarray.DataArray 'x' (time: 61000)>
array([ 757.52169836,  774.25570624,  790.90617316, ..., 1451.51121823,
       1456.44331123, 1461.08406233])
Coordinates:
    lidar_id  <U7 'koshava'
  * time      (time) float64 0.0 1.0 2.0 3.0 ... 6.17e+04 6.17e+04 6.17e+04
Attributes:
    long_name:      X coordinate
    standard_name:  projection_x_coordinate
    units:          m
    URI:

In [17]:
merged_ds = temp.merge(temp2)

In [19]:
tmp = 5
tmp+=1

tmp

6

In [27]:
temp2.az.interp(time=np.nan)

KeyError: nan